# Algorithm 25: makeRotX

Creates a rotation matrix for rotation around the x-axis by a given angle. This is a fundamental building block for constructing sidechain conformations from torsion (dihedral) angles.

## Algorithm Pseudocode

![makeRotX](../imgs/algorithms/makeRotX.png)

## Source Code Location
- **File**: `AF2-source-code/model/all_atom.py`
- **Usage**: Within `torsion_angles_to_frames`

## Mathematical Definition

Rotation around the x-axis by angle θ:

$$R_x(\theta) = \begin{pmatrix} 1 & 0 & 0 \\ 0 & \cos\theta & -\sin\theta \\ 0 & \sin\theta & \cos\theta \end{pmatrix}$$

This rotates points in the y-z plane while keeping the x-coordinate fixed.

In [ ]:
import numpy as np

np.random.seed(42)

## NumPy Implementation

In [ ]:
def make_rot_x(angle):
    """
    makeRotX - Algorithm 25.
    
    Creates rotation matrix around x-axis.
    
    Args:
        angle: Rotation angle in radians (scalar or array)
    
    Returns:
        R: Rotation matrix [..., 3, 3]
    """
    angle = np.asarray(angle)
    c = np.cos(angle)
    s = np.sin(angle)
    
    # Build rotation matrix
    R = np.zeros(angle.shape + (3, 3))
    R[..., 0, 0] = 1.0
    R[..., 1, 1] = c
    R[..., 1, 2] = -s
    R[..., 2, 1] = s
    R[..., 2, 2] = c
    
    return R


def make_rot_y(angle):
    """
    Rotation around y-axis (for completeness).
    
    R_y(θ) = [[cos, 0, sin], [0, 1, 0], [-sin, 0, cos]]
    """
    angle = np.asarray(angle)
    c = np.cos(angle)
    s = np.sin(angle)
    
    R = np.zeros(angle.shape + (3, 3))
    R[..., 0, 0] = c
    R[..., 0, 2] = s
    R[..., 1, 1] = 1.0
    R[..., 2, 0] = -s
    R[..., 2, 2] = c
    
    return R


def make_rot_z(angle):
    """
    Rotation around z-axis (for completeness).
    
    R_z(θ) = [[cos, -sin, 0], [sin, cos, 0], [0, 0, 1]]
    """
    angle = np.asarray(angle)
    c = np.cos(angle)
    s = np.sin(angle)
    
    R = np.zeros(angle.shape + (3, 3))
    R[..., 0, 0] = c
    R[..., 0, 1] = -s
    R[..., 1, 0] = s
    R[..., 1, 1] = c
    R[..., 2, 2] = 1.0
    
    return R

## Test Examples

In [ ]:
# Test 1: Identity rotation (angle = 0)
print("Test 1: Identity Rotation (θ = 0)")
print("="*60)

R0 = make_rot_x(0.0)
print(f"R_x(0):")
print(R0)
print(f"\nIs identity: {np.allclose(R0, np.eye(3))}")

In [ ]:
# Test 2: 90-degree rotations
print("\nTest 2: 90° Rotations")
print("="*60)

R90_x = make_rot_x(np.pi/2)
R90_y = make_rot_y(np.pi/2)
R90_z = make_rot_z(np.pi/2)

print(f"R_x(90°):")
print(np.round(R90_x, 3))

print(f"\nR_y(90°):")
print(np.round(R90_y, 3))

print(f"\nR_z(90°):")
print(np.round(R90_z, 3))

In [ ]:
# Test 3: Verify rotation behavior
print("\nTest 3: Rotation Behavior")
print("="*60)

# A point on the y-axis rotated 90° around x should go to z-axis
point_y = np.array([0, 1, 0])
R90 = make_rot_x(np.pi/2)
rotated = R90 @ point_y

print(f"Point on y-axis: {point_y}")
print(f"After 90° rotation around x: {np.round(rotated, 3)}")
print(f"Expected: [0, 0, 1]")
print(f"Correct: {np.allclose(rotated, [0, 0, 1])}")

In [ ]:
# Test 4: Batch rotation
print("\nTest 4: Batch Rotation")
print("="*60)

angles = np.array([0, np.pi/4, np.pi/2, np.pi])
R_batch = make_rot_x(angles)

print(f"Input angles: {angles}")
print(f"Output shape: {R_batch.shape}")

# Verify each is a valid rotation
for i, a in enumerate(angles):
    R = R_batch[i]
    is_ortho = np.allclose(R @ R.T, np.eye(3))
    det = np.linalg.det(R)
    print(f"  θ={a:.2f}: orthogonal={is_ortho}, det={det:.4f}")

In [ ]:
# Test 5: Composition of rotations
print("\nTest 5: Rotation Composition")
print("="*60)

# Two 45° rotations should equal one 90° rotation
R45 = make_rot_x(np.pi/4)
R90 = make_rot_x(np.pi/2)

R_composed = R45 @ R45

print(f"R_x(45°) @ R_x(45°) = R_x(90°)?")
print(f"Match: {np.allclose(R_composed, R90)}")

In [ ]:
# Test 6: Application to chi angles (sidechain)
print("\nTest 6: Chi Angle Application (Sidechain Building)")
print("="*60)

# Simulate building a sidechain with chi1 and chi2 rotations
chi1 = np.pi / 3  # 60° chi1 angle
chi2 = np.pi / 6  # 30° chi2 angle

# Build rotation frames
R_chi1 = make_rot_x(chi1)
R_chi2 = make_rot_x(chi2)

# Combined rotation
R_combined = R_chi1 @ R_chi2

print(f"Chi1 angle: {np.degrees(chi1):.1f}°")
print(f"Chi2 angle: {np.degrees(chi2):.1f}°")

# Place an atom along the chain
atom_local = np.array([1.5, 0, 0])  # 1.5Å bond

# Apply rotations (in proper frame)
atom_after_chi1 = R_chi1 @ atom_local
atom_after_chi2 = R_combined @ atom_local

print(f"\nAtom local position: {atom_local}")
print(f"After chi1 rotation: {np.round(atom_after_chi1, 3)}")
print(f"After chi1 + chi2: {np.round(atom_after_chi2, 3)}")

## Verification: Key Properties

In [ ]:
print("Verification: Key Properties")
print("="*60)

# Random angles
angles = np.random.uniform(-np.pi, np.pi, size=100)

for i, theta in enumerate(angles[:5]):
    R = make_rot_x(theta)
    
    # Property 1: Orthogonal (R @ R.T = I)
    ortho = np.allclose(R @ R.T, np.eye(3))
    
    # Property 2: Determinant = 1 (proper rotation)
    det = np.linalg.det(R)
    det_one = np.allclose(det, 1.0)
    
    # Property 3: x-axis is fixed
    x_fixed = np.allclose(R @ np.array([1, 0, 0]), [1, 0, 0])
    
    print(f"θ={theta:+.3f}: ortho={ortho}, det=1: {det_one}, x-fixed: {x_fixed}")

# Test all angles
all_ortho = all(np.allclose(make_rot_x(a) @ make_rot_x(a).T, np.eye(3)) for a in angles)
all_det_one = all(np.allclose(np.linalg.det(make_rot_x(a)), 1.0) for a in angles)

print(f"\nAll 100 angles:")
print(f"  All orthogonal: {all_ortho}")
print(f"  All det=1: {all_det_one}")

In [ ]:
# Property: Inverse is R(-θ)
print("\nProperty: R(-θ) = R(θ)^(-1)")
print("="*60)

theta = np.random.uniform(-np.pi, np.pi)
R_pos = make_rot_x(theta)
R_neg = make_rot_x(-theta)

# R(θ) @ R(-θ) should be identity
product = R_pos @ R_neg
print(f"R({theta:.3f}) @ R({-theta:.3f}):")
print(np.round(product, 6))
print(f"Is identity: {np.allclose(product, np.eye(3))}")

## Application in Sidechain Building

In AlphaFold2, torsion angles (chi angles) define rotations around bonds:

| Angle | Bond | Description |
|-------|------|-------------|
| χ1 | CA-CB | First sidechain rotation |
| χ2 | CB-CG | Second sidechain rotation |
| χ3 | CG-CD | Third sidechain rotation |
| χ4 | CD-CE | Fourth sidechain rotation |

The `make_rot_x` function is used because the torsion angle is defined as rotation around the bond axis, which is aligned with x in the local frame.

In [ ]:
# Demonstrate sidechain building
print("Sidechain Building Example")
print("="*60)

# Lysine has 4 chi angles
chi_angles = [np.radians(60), np.radians(180), np.radians(180), np.radians(60)]

# Bond lengths (Angstroms)
bond_lengths = [1.53, 1.52, 1.52, 1.52]  # CB-CG, CG-CD, CD-CE, CE-NZ

# Start from CB position
positions = [np.array([0, 0, 0])]  # CB at origin
current_frame = np.eye(3)

for i, (chi, bond) in enumerate(zip(chi_angles, bond_lengths)):
    # Apply chi rotation
    R_chi = make_rot_x(chi)
    current_frame = current_frame @ R_chi
    
    # Move along bond
    bond_vector = current_frame @ np.array([bond, 0, 0])
    new_pos = positions[-1] + bond_vector
    positions.append(new_pos)
    
    print(f"χ{i+1}={np.degrees(chi):.0f}°: atom at {np.round(new_pos, 2)}")

# End-to-end distance
e2e = np.linalg.norm(positions[-1] - positions[0])
print(f"\nEnd-to-end distance: {e2e:.2f}Å")

## Source Code Reference

```python
# From AF2-source-code/model/all_atom.py

def torsion_angles_to_frames(aatype, backb_to_global, torsion_angles_sin_cos):
  """Build sidechain frames from torsion angles."""
  
  # Extract sin and cos
  sin_angles = torsion_angles_sin_cos[..., 0]
  cos_angles = torsion_angles_sin_cos[..., 1]
  
  # Build rotation matrices (Algorithm 25)
  # R_x(θ) = [[1, 0, 0], [0, cos, -sin], [0, sin, cos]]
  # But uses sin/cos directly without computing angle
  
  # For each chi angle, compose rotations
  for chi_idx in range(num_chi):
    chi_rot = make_rotation_matrix(sin_angles[..., chi_idx], 
                                    cos_angles[..., chi_idx])
    current_frame = compose_frames(current_frame, chi_rot)
```

## Key Insights

1. **Fundamental Building Block**: `make_rot_x` is the basic operation for building sidechain conformations.

2. **Torsion Angle Rotation**: Torsion angles are rotations around bonds, naturally expressed as x-axis rotations in local frames.

3. **Sin/Cos Representation**: AlphaFold2 predicts sin and cos of angles rather than angles directly, avoiding periodicity issues.

4. **Composition**: Multiple chi rotations are composed to build the full sidechain.

5. **SO(3) Rotation**: The output is always a proper rotation matrix (orthogonal, det=1).

6. **Vectorized**: The implementation supports batch processing of angles.